In [203]:
## Imports
import os
from pathlib import Path
import numpy as np
import pandas as pd
from typing import Tuple, Dict

import polars as pl

import warnings
warnings.filterwarnings('ignore')

In [204]:
## Configuration and Data Loading (kaggle_evaluation only)
# import kaggle_evaluation.default_inference_server as kdeval
# DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')

## Configuration and Data Loading (local version only)
DATA_DIR = Path("01_data")

# Read CSV files from data_path
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH  = DATA_DIR / 'test.csv'

VALIDATION_SIZE = 2700          # days, approx. 30% of data
# RANDOM_SEED = 42

import random
RANDOM_SEED = random.randint(1, 10000)

VOL_MULTIPLIER_LIMIT = 1.2
VOL_WINDOW = 20

def time_split_train_val(df: pd.DataFrame, val_size: int = 2700):
    df = df.sort_values('date_id').reset_index(drop=True)
    train_df = df.iloc[:-val_size].copy()
    val_df   = df.iloc[-val_size:].copy()
    return train_df, val_df

train_raw = pd.read_csv(TRAIN_PATH)
test_raw  = pd.read_csv(TEST_PATH)

# lower column names
train_raw.columns = [c.lower() for c in train_raw.columns]
test_raw.columns  = [c.lower() for c in test_raw.columns]
train_raw.shape, test_raw.shape

((8990, 98), (10, 99))

In [205]:
main_features = ['m4', 'm11', 'v13', 's2', 'p6', 'm12', 's1', 'v9', 'e17', 'p7'] # from EDA

# keep only main features in train_raw
train_p = train_raw[['date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns'] + main_features].copy()
test_p  = test_raw[['date_id'] + main_features].copy()

In [206]:
train_p.shape

(8990, 14)

In [207]:
train_p.head()

,date_id,forward_returns,risk_free_rate,market_forward_excess_returns,m4,m11,v13,s2,p6,m12,s1,v9,e17,p7
0,0,-0.002421,0.000301,-0.003038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-0.008495,0.000303,-0.009114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-0.009624,0.000301,-0.010243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.004662,0.000299,0.004046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,-0.011686,0.000299,-0.012301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
# check for nan in train_p
train_p.isna().sum()

date_id                             0
forward_returns                     0
risk_free_rate                      0
market_forward_excess_returns       0
m4                               1006
m11                              1006
v13                              1511
s2                               1006
p6                               1638
m12                              1006
s1                               1006
v9                               4539
e17                              1006
p7                               1616
dtype: int64

In [209]:
## Feature Preparation
excluded = {'date_id', 'forward_returns', 'risk_free_rate', 'market_forward_excess_returns'}
feature_cols = [c for c in train_p.columns if c not in excluded]
feature_cols = [c for c in feature_cols if c in test_raw.columns]

In [210]:
feature_cols

['m4', 'm11', 'v13', 's2', 'p6', 'm12', 's1', 'v9', 'e17', 'p7']

In [211]:
def prepare_df(df: pd.DataFrame, median_map: Dict[str, float], feature_cols: list) -> pd.DataFrame:
    df = df.copy()
    for c in feature_cols:
        if c not in df.columns:
            df[c] = 0.0
            df[f'{c}_was_na'] = 1
            continue
        if df[c].dtype.kind in 'fiu':
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
        else:
            df[c] = pd.to_numeric(df[c], errors='coerce')
            med = median_map.get(c, 0.0)
            was_na = df[c].isna().astype(int)
            df[c] = df[c].fillna(med)
            df[f'{c}_was_na'] = was_na
    return df

In [212]:
## Train / Validation Split and Median Imputation
train_df, val_df = time_split_train_val(train_raw, val_size=VALIDATION_SIZE)

median_map = {c: float(train_df[c].median(skipna=True)) if train_df[c].dtype.kind in 'fiu' else 0.0 
              for c in feature_cols}

train_p = prepare_df(train_df, median_map, feature_cols)
val_p   = prepare_df(val_df, median_map, feature_cols)
test_p  = prepare_df(test_raw, median_map, feature_cols)

final_features = [f for c in feature_cols for f in (c, f"{c}_was_na")]
print("Number of features:", len(final_features))

Number of features: 20


In [213]:
train_p.head()

,date_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,...,m4_was_na,m11_was_na,v13_was_na,s2_was_na,p6_was_na,m12_was_na,s1_was_na,v9_was_na,e17_was_na,p7_was_na
0,0,0,0,0,1,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
1,1,0,0,0,1,1,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,1,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
3,3,0,0,0,1,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,4,0,0,0,1,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [214]:
print(train_p.columns)

Index(['date_id', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9',
       ...
       'm4_was_na', 'm11_was_na', 'v13_was_na', 's2_was_na', 'p6_was_na',
       'm12_was_na', 's1_was_na', 'v9_was_na', 'e17_was_na', 'p7_was_na'],
      dtype='object', length=108)


In [215]:
final_features

['m4',
 'm4_was_na',
 'm11',
 'm11_was_na',
 'v13',
 'v13_was_na',
 's2',
 's2_was_na',
 'p6',
 'p6_was_na',
 'm12',
 'm12_was_na',
 's1',
 's1_was_na',
 'v9',
 'v9_was_na',
 'e17',
 'e17_was_na',
 'p7',
 'p7_was_na']

In [216]:
# === Bayesian Ridge Training ===
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error

# Prepara i dati (usando gli stessi DataFrame)
X_train = train_p[final_features]
y_train = train_p['forward_returns']
X_val   = val_p[final_features]
y_val   = val_p['forward_returns']

# Check for NaN values and handle them
print(f"NaN values in X_train: {X_train.isnull().sum().sum()}")
print(f"NaN values in y_train: {y_train.isnull().sum()}")

# Fill any remaining NaN values
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)
X_val = X_val.fillna(0)
y_val = y_val.fillna(0)

# count nan in X_train, y_train, X_val, y_val
print(f"After filling NaN values:")
print(f"NaN values in X_train: {X_train.isnull().sum().sum()}")
print(f"NaN values in y_train: {y_train.isnull().sum()}")
print(f"NaN values in X_val: {X_val.isnull().sum().sum()}")
print(f"NaN values in y_val: {y_val.isnull().sum()}")

NaN values in X_train: 0
NaN values in y_train: 0
After filling NaN values:
NaN values in X_train: 0
NaN values in y_train: 0
NaN values in X_val: 0
NaN values in y_val: 0


In [217]:
# Crea e allena il modello
model = BayesianRidge()
model.fit(X_train, y_train)

# Valutazione sul validation set
val_pred = model.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, val_pred))
print(f"Validation RMSE: {rmse_val:.6f}")

Validation RMSE: 0.010211


In [218]:
# === Volatility Scaling Calibration (BayesianRidge version) ===
def strategy_stats(returns, exposures):
    """Compute Sharpe and volatility for a given exposure series."""
    strat = exposures * returns
    mean = np.nanmean(strat)
    std  = np.nanstd(strat)
    sharpe = (mean / (std + 1e-9)) * np.sqrt(252)
    vol = std * np.sqrt(252)
    return {'sharpe': sharpe, 'vol': vol}

# Ensure validation data has no NaN values
val_features_clean = val_p[final_features].fillna(0)

# Predict mean and std (Bayesian posterior uncertainty)
val_pred_mean, val_pred_std = model.predict(val_features_clean, return_std=True)

# Define market volatility for scaling reference
market_vol = np.nanstd(train_p['forward_returns']) * np.sqrt(252)
print(f"Market annualized volatility: {market_vol:.4f}")

# We'll use a dynamic confidence weight: confidence = 1 / (1 + std)
confidence = 1 / (1 + val_pred_std)
val_conf_adj = val_pred_mean * confidence  # lower exposure when uncertainty is high

# Grid search for best scaling factor k (Sharpe ratio under vol constraint)
best_k, best_sharpe = 0.1, -1e9
for k in np.linspace(0.01, 5.0, 100):
    exposures = np.clip((k * val_conf_adj), 0, 2)
    stats = strategy_stats(val_p['forward_returns'], exposures)
    if stats['vol'] <= VOL_MULTIPLIER_LIMIT * market_vol and stats['sharpe'] > best_sharpe:
        best_k = k
        best_sharpe = stats['sharpe']

print(f"Chosen scaling factor k={best_k:.3f} | Validation Sharpe={best_sharpe:.2f}")

Market annualized volatility: 0.1698
Chosen scaling factor k=5.000 | Validation Sharpe=0.88


In [219]:
## Test Predictions + Smoothing
test_features_clean = test_p[final_features].fillna(0)
test_pred = model.predict(test_features_clean)

alpha = 0.8
smoothed_allocation = []
prev = 0.0
for x in np.clip(best_k * test_pred, 0, 2):
    s = alpha * x + (1 - alpha) * prev
    smoothed_allocation.append(s)
    prev = s
smoothed_allocation = np.array(smoothed_allocation)

# replace in final submission
submission_df = pd.DataFrame({
    'date_id': test_p['date_id'],
    'prediction': smoothed_allocation  
})
# submission_df.to_csv("submission_br.csv", index=False)
print("Saved submission_br.csv")

Saved submission_br.csv


In [220]:
"""
Kaggle Evaluation Metric:

strategy_returns = risk_free_rate * (1 - position) + position * forward_returns

if position = 0 → invest in risk-free asset,

if position = 1 → invest like the market,

if position = 2 → you are leveraged ×2 on the market.


def score():

strategy_returns = rf * (1 - pos) + pos * fwd_returns

In the code, the calibration seeks the best Sharpe of the portfolio exposed to pos by calculating:

strat = exposures * returns
""";

In [221]:
# # === Kaggle Inference Server Wrapper (BayesianRidge version) ===

# _model = model                     # BayesianRidge fitted model
# _best_k = best_k                   # scaling factor from validation calibration
# _history_returns = list(train_p['forward_returns'].iloc[-VOL_WINDOW:].tolist())

# def predict(pl_df: pl.DataFrame) -> float:
#     """Predict allocation for one timestep (Kaggle competition API)."""
#     global _history_returns
#     # Convert Polars → Pandas
#     pdf = pl_df.to_pandas()
    
#     # Apply same preprocessing
#     pdf_p = prepare_df(pdf, median_map, feature_cols)
#     for f in final_features:
#         if f not in pdf_p.columns:
#             pdf_p[f] = 0.0
    
#     # Convert to NumPy and predict
#     x = pdf_p[final_features].to_numpy()

#     """
#     added standard deviation based confidence adjustment
#     to reduce allocation when uncertainty is high
#     """

#     pred, std = model.predict(x, return_std=True)
    
#     # Compute rolling volatility estimate
#     vol_est = np.std(_history_returns) or 1e-3
#     confidence = 1 / (1 + std)
#     alloc = float(np.clip(_best_k * pred * confidence / (vol_est + 1e-9), 0, 2))
    
#     # Update history (for volatility tracking)
#     if 'lagged_forward_returns' in pl_df.columns:
#         try:
#             _history_returns.append(float(pl_df['lagged_forward_returns'][0]))
#         except:
#             _history_returns.append(0.0)
#     else:
#         _history_returns.append(0.0)
    
#     # Keep only the last VOL_WINDOW entries
#     _history_returns = _history_returns[-VOL_WINDOW:]
#     return alloc

# # Instantiate the Kaggle inference server
# server = kdeval.DefaultInferenceServer(predict)

# if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#     server.serve()
# else:
#     server.run_local_gateway((str(DATA_DIR),))
